In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%cd /content/gdrive/My Drive/TransferLearning
!ls

/content/gdrive/My Drive/TransferLearning
X_test.pickle  X_train.pickle  Y_test.pickle  Y_train.pickle


In [0]:
%tensorflow_version 1.x

In [0]:
import pickle

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

In [0]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
import keras.optimizers
from sklearn.metrics import classification_report
import keras.optimizers
from keras.applications import vgg16
import numpy as np

Using TensorFlow backend.


In [0]:
tensorboard = TensorBoard(log_dir='./VGG16logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)


In [0]:
vgg16_model = vgg16.VGG16()
vgg16_model.summary()

type(vgg16_model)











Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)   

keras.engine.training.Model

In [0]:
model1 = Sequential()
#for layer in vgg16_model.layers:
for layer in vgg16_model.layers[:-3]: #model1.layers.pop() not working, so -1
    model1.add(layer) # convert to sequantial model
    
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
for layer in model1.layers:
    layer.trainable = False
    
model1.add(Dense(4,activation = 'softmax'))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
model1.compile(loss='sparse_categorical_crossentropy',
              optimizer= "adam",
              metrics=['accuracy'],
              )

In [0]:
#Fit the model
            model1.fit(X_train, Y_train,
            batch_size=32,
            epochs=20,
            validation_data=(X_test,Y_test),
            callbacks=[tensorboard])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1006 samples, validate on 607 samples


Epoch 1/20
1006/1006 [==============================] - 20s 20ms/step - loss: 3.8662 - acc: 0.6978 - val_loss: 2.2545 - val_acc: 0.8221

Epoch 2/20
1006/1006 [==============================] - 9s 9ms/step - loss: 2.4231 - acc: 0.8310 - val_loss: 2.4195 - val_acc: 0.8221
Epoch 3/20
1006/1006 [==============================] - 9s 9ms/step - loss: 2.1622 - acc: 0.8539 - val_loss: 1.9238 - val_acc: 0.8534
Epoch 4/20
1006/1006 [==============================] - 9s 9ms/step - loss: 1.7979 - acc: 0.8767 - val_loss: 1.7131 - val_acc: 0.8764
Epoch 5/20
1006/1006 [==============================] - 9s 9ms/step - loss: 1.9262 - acc: 0.8708 - val_loss: 2.6435 - val_acc: 0.8287
Epoch 6/20
1006/1006 [==============================] - 9s 9ms/step - loss: 1.7695 - acc: 0.8797 - val_loss: 1.6436 - val_acc: 0.8649
Epoch 7/20
1006/1006 [==========================

In [0]:
#Score 
scores = model1.evaluate(X_test, Y_test, verbose=1) 
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

607/607 [==============================] - 4s 6ms/step
Test loss: 0.8918053124351596
Test accuracy: 0.9390444795814338


In [0]:
model1.save("VGG16model94accuracy.h5")

In [0]:
 #precision    recall  f1-score   support
y_pred = model1.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_bool))

607/607 [==============================] - 6s 10ms/step
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       150
           1       0.97      0.85      0.90       150
           2       0.96      0.96      0.96       157
           3       0.94      0.95      0.94       150

    accuracy                           0.94       607
   macro avg       0.94      0.94      0.94       607
weighted avg       0.94      0.94      0.94       607

